In [1]:
import os
import sys
import pandas as pd 
import pingouin as pg 

import torch

parent_dir = os.path.dirname(os.getcwd())
src_dir = os.path.join(parent_dir, 'src')


sys.path.insert(0, src_dir)
import read
import evaluation.pearson_emotion


In [4]:
pd.read_csv('C:/Users/670302093/OneDrive - Signify/ClipSoundSymbolism/results/pearson/emotion_pearson.csv')

,Emotion,Type,Sample Size (n),Pearson Correlation (r),95% Confidence Interval,P-value,Bayes Factor,Statistical Power
0,Anger,real,68,0.280378,[0.04 0.49],0.020566,2.091,0.647139
1,Disgust,real,68,0.272289,[0.04 0.48],0.024682,1.790,0.620396
2,Fear,real,68,0.043297,[-0.2 0.28],0.725903,0.161,0.064030
3,Joy,real,68,-0.196324,[-0.42 0.04],0.108588,0.534,0.364652
4,Sadness,real,68,0.254194,[0.02 0.46],0.036460,1.289,0.559106
5,Surprise,real,68,-0.152219,[-0.38 0.09],0.215274,0.321,0.237676
6,Anger,pseudowords,272,0.033615,[-0.09 0.15],0.580954,0.088,0.085563
7,Disgust,pseudowords,272,0.039559,[-0.08 0.16],0.515906,0.094,0.099594
8,Fear,pseudowords,272,-0.007979,[-0.13 0.11],0.895782,0.077,0.051911
9,Joy,pseudowords,272,-0.026904,[-0.15 0.09],0.658671,0.084,0.072609


In [4]:
!pip install --upgrade pybind11


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      c:\Users\670302093\OneDrive - Signify\ClipSoundSymbolism\.venv\Scripts\python.exe: No module named pip
      Traceback (most recent call last):
        File "<string>", line 38, in __init__
      ModuleNotFoundError: No module named 'pybind11'
      
      During handling of the above exception, another exception occurred:
      
      Traceback (most recent call last):
        File "c:\Users\670302093\OneDrive - Signify\ClipSoundSymbolism\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\670302093\OneDrive - Signify\ClipSoundSymbolism\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^